In [1]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

ColumnNames = ["startDate", "endDate", "value"]

globalPath = "/home/chowder/Documents/personalData/ouraRing/"
jsonFileName = "oura_2021-01-16T02-12-49.json"

parquetName = "ouraHypnoDf.parquet.gzip"

with open(globalPath + jsonFileName) as f:
  jsonData = json.load(f)

1.1.2


In [2]:
def ouraSleepStageToStandard(ouraNum):
    if ouraNum == 4: #awake
        return 0
    if ouraNum == 2: #light sleep
        return 1
    if ouraNum == 1: #deep sleep
        return 2
    if ouraNum == 3: #REM sleep
        return 3
    

In [3]:
print(jsonData['sleep'][0]['bedtime_start'])
print(jsonData['sleep'][0]['bedtime_end'])
print(jsonData['sleep'][0]['hypnogram_5min'])
print(len(jsonData['sleep'][0]['hypnogram_5min']))
print(pd.to_datetime(jsonData['sleep'][0]['bedtime_start']) + datetime.timedelta(minutes=len(jsonData['sleep'][0]['hypnogram_5min'])*5))


2020-10-15T01:03:59-07:00
2020-10-15T09:47:59-07:00
424221114322222121112211123322211111122222211223312224333333242221122222333333442222222223333344444444444
105
2020-10-15 09:48:59-07:00


In [4]:
listRecords = []
startTime = pd.to_datetime(jsonData['sleep'][0]['bedtime_start'])

hypnogram_5min = jsonData['sleep'][0]['hypnogram_5min']
toadd = []
toadd.append(startTime)

for i in range(1, len(hypnogram_5min)):
    if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
        toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
        toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1])))
        listRecords.append(toadd)
        toadd = [startTime + datetime.timedelta(minutes=i*5)]

toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
listRecords.append(toadd)

In [5]:
listRecords[-1]

[Timestamp('2020-10-15 08:53:59-0700', tz='pytz.FixedOffset(-420)'),
 Timestamp('2020-10-15 09:48:59-0700', tz='pytz.FixedOffset(-420)'),
 0]

In [27]:
listRecords = []

for SleepJSON in jsonData['sleep']:
    startTime = pd.to_datetime(SleepJSON['bedtime_start']).tz_convert("US/Arizona")

    hypnogram_5min = SleepJSON['hypnogram_5min']
    toadd = []
    toadd.append(startTime)

    for i in range(1, len(hypnogram_5min)):
        if(hypnogram_5min[i-1] != hypnogram_5min[i]):#if the block has ended
            toadd.append(startTime + datetime.timedelta(minutes=i*5)) # add the end time
            toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i-1])))
            listRecords.append(toadd)
            toadd = [startTime + datetime.timedelta(minutes=i*5)]
    
    toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)*5))
    toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[-1])))
    listRecords.append(toadd)




In [28]:
len(listRecords)

2424

In [29]:
for restfulJSON in jsonData['restful_periods']:
    startTime = pd.to_datetime(restfulJSON['bedtime_start']).tz_convert("US/Arizona")
    endTime = pd.to_datetime(restfulJSON['bedtime_end']).tz_convert("US/Arizona")
    listRecords.append([startTime, endTime, 1]) #fill in naps with light sleep

In [30]:
len(listRecords)

2511

In [31]:
listRecords = sorted(listRecords, key=lambda x:x[0])


In [32]:
# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(listRecords)-1):
    filledRecords.append(listRecords[ri])
    # if there is a gap between this rows end date and the next rows start date
    if listRecords[ri][1] < listRecords[ri+1][0]:
        filledRecords.append([
            listRecords[ri][1],
            listRecords[ri+1][0],
            -1
        ])
print(len(filledRecords))

2684


In [33]:
set([x[1] for x in filledRecords])

{Timestamp('2020-11-20 06:23:26-0700', tz='US/Arizona'),
 Timestamp('2020-10-16 05:01:23-0700', tz='US/Arizona'),
 Timestamp('2020-10-20 14:13:32-0700', tz='US/Arizona'),
 Timestamp('2020-11-22 05:12:01-0700', tz='US/Arizona'),
 Timestamp('2021-01-05 03:09:07-0700', tz='US/Arizona'),
 Timestamp('2020-11-30 08:12:59-0700', tz='US/Arizona'),
 Timestamp('2020-11-10 06:25:29-0700', tz='US/Arizona'),
 Timestamp('2021-01-10 07:00:49-0700', tz='US/Arizona'),
 Timestamp('2020-11-14 09:04:29-0700', tz='US/Arizona'),
 Timestamp('2020-10-30 09:08:00-0700', tz='US/Arizona'),
 Timestamp('2020-11-20 08:28:26-0700', tz='US/Arizona'),
 Timestamp('2021-01-10 03:10:49-0700', tz='US/Arizona'),
 Timestamp('2020-10-30 07:13:00-0700', tz='US/Arizona'),
 Timestamp('2020-10-16 03:16:23-0700', tz='US/Arizona'),
 Timestamp('2020-10-30 03:23:00-0700', tz='US/Arizona'),
 Timestamp('2020-10-24 03:00:53-0700', tz='US/Arizona'),
 Timestamp('2021-01-06 07:37:21-0700', tz='US/Arizona'),
 Timestamp('2020-12-10 04:18:28

In [34]:
ouraHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

ouraHypnoDf = ouraHypnoDf.set_index("startDate")
ouraHypnoDf

,endDate,value
startDate,,
2020-10-15 01:03:59-07:00,2020-10-15 01:08:59-07:00,0
2020-10-15 01:08:59-07:00,2020-10-15 01:13:59-07:00,1
2020-10-15 01:13:59-07:00,2020-10-15 01:18:59-07:00,0
2020-10-15 01:18:59-07:00,2020-10-15 01:28:59-07:00,1
2020-10-15 01:28:59-07:00,2020-10-15 01:43:59-07:00,2
...,...,...
2021-01-15 06:56:19-07:00,2021-01-15 07:01:19-07:00,3
2021-01-15 07:01:19-07:00,2021-01-15 07:41:19-07:00,1
2021-01-15 07:41:19-07:00,2021-01-15 08:26:19-07:00,3


In [35]:
ouraHypnoDf.dtypes

endDate    datetime64[ns, US/Arizona]
value                           int64
dtype: object

In [36]:
ouraHypnoDf["endDate"]

startDate
2020-10-15 01:03:59-07:00   2020-10-15 01:08:59-07:00
2020-10-15 01:08:59-07:00   2020-10-15 01:13:59-07:00
2020-10-15 01:13:59-07:00   2020-10-15 01:18:59-07:00
2020-10-15 01:18:59-07:00   2020-10-15 01:28:59-07:00
2020-10-15 01:28:59-07:00   2020-10-15 01:43:59-07:00
                                       ...           
2021-01-15 06:56:19-07:00   2021-01-15 07:01:19-07:00
2021-01-15 07:01:19-07:00   2021-01-15 07:41:19-07:00
2021-01-15 07:41:19-07:00   2021-01-15 08:26:19-07:00
2021-01-15 08:26:19-07:00   2021-01-15 08:31:19-07:00
2021-01-15 08:31:19-07:00   2021-01-15 08:51:19-07:00
Name: endDate, Length: 2684, dtype: datetime64[ns, US/Arizona]

In [37]:
ouraHypnoDf.to_parquet(globalPath + parquetName,
              compression='gzip') 